In [1]:
#from google.colab import files
#uploaded = files.upload()

Saving cleanedData.csv to cleanedData.csv


In [26]:
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pickle

data = pd.read_csv("cleanedData.csv")
data = data.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# Convert categorical columns to numerical using label encoding
data["Synopsis"].fillna("", inplace=True)
le = LabelEncoder()
data["Type"] = le.fit_transform(data["Type"])
data["Producer"] = le.fit_transform(data["Producer"])
data["Studio"] = le.fit_transform(data["Studio"])
data["Genre"] = le.fit_transform(data["Genre"])
# Combine numerical and categorical features
X_text = data[["Title", "Synopsis"]]
X_num = data[["Type", "Producer", "Studio","Genre"]].fillna(data.median())
y = data["Rating"]
# Train Word2Vec model
sentences = [text.split() for text in data["Title"] + data["Synopsis"]]
w2v_model = Word2Vec(sentences, size=100, window=5, min_count=1, workers=4)

X_text1 = []
for index, row in X_text.iterrows():
    title_vector = np.mean([w2v_model.wv[word] for word in row["Title"] if word in w2v_model.wv], axis=0)
    if np.isnan(title_vector).any():
        title_vector = np.zeros(w2v_model.vector_size)
    synopsis_vector = np.mean([w2v_model.wv[word] for word in row["Synopsis"] if word in w2v_model.wv], axis=0)
    if np.isnan(synopsis_vector).any():
        synopsis_vector = np.zeros(w2v_model.vector_size)
    X_text1.append(np.concatenate((title_vector, synopsis_vector)))

X_text1 = np.array(X_text1)

<ipython-input-26-a1ce593d40b9>:25: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_num = data[["Type", "Producer", "Studio","Genre"]].fillna(data.median())
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [31]:
# Combine numerical and textual data
X = np.concatenate((X_text1, X_num), axis=1)
X = np.nan_to_num(X, nan=np.nan)

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

# Replace missing or infinite values with the mean of the column
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
# Replace missing or infinite values with the mean of the column
X = np.nan_to_num(X, nan=np.nanmean(X, axis=0), posinf=np.nan, neginf=np.nan)
# Remove rows containing missing or infinite values
X = X[~np.isnan(X).any(axis=1)]
y = y[~np.isnan(X).any(axis=1)]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
mask = np.isfinite(y_train)
X_train = X_train[mask]
y_train = y_train[mask]
# Create a random forest regressor model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [49]:
# Evaluate the model on the test data
np.nan_to_num(X_test, nan=np.nanmean(X, axis=0), posinf=np.nan, neginf=np.nan)
mask = np.isfinite(y_test)
X_test = X_test[mask]
y_test = y_test[mask]
score = rf.score(X_test, y_test)
print("R^2 score:", score)

R^2 score: 0.9739633021312945


In [50]:
# Make predictions on the test data
y_pred = rf.predict(X_test)
print(y_test)

23784    6.28
34000    7.84
38994    6.63
35632    7.12
19811    7.91
         ... 
24324    6.81
33350    7.40
35128    8.42
8246     6.72
31150    5.71
Name: Rating, Length: 9475, dtype: float64


In [51]:
print(y_pred)

[6.28   7.84   6.6508 ... 8.42   6.72   6.0518]


In [52]:
# save the model to disk using pickle
filename = 'modelRDF.pkl'
with open(filename, 'wb') as file:
    pickle.dump(rf, file)